In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
os.chdir('/content/drive/My Drive/ProyectoIA2/generative-spectral-datasets')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import numpy as np
from models.main import make_autoencoder, make_discriminator, spectralGAN
from models.utils import discriminator_loss, autoencoder_loss
from models.metrics import PSNR, SSIM
from data import load_dataset

import matplotlib.pyplot as plt

input_shape = (512,512,31)

In [3]:
BATCH_SIZE = 3

train_params = dict(
    batch_size = BATCH_SIZE,
    shuffle = True,
    cache = True
)

test_params = dict(
    batch_size = BATCH_SIZE,
    cache = True
)

train_ds , test_ds = load_dataset( train_params = train_params, test_params = test_params)

In [4]:
a_optimizer = tf.keras.optimizers.Adam(1e-4)
d_optimizer = tf.keras.optimizers.Adam(1e-4)

FACTOR = [ 1 , 1 , 1/2 , 1/2 , 1/4, 1/8]
features= 64

autoencoder = make_autoencoder(input_shape, features=features, factors=FACTOR)
discriminator = make_discriminator(input_shape)

In [5]:
autoencoder.summary()
discriminator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 31)]    0         
_________________________________________________________________
encoder (Encoder)            (None, 512, 512, 8)       59184     
_________________________________________________________________
generator (Generator)        (None, 512, 512, 31)      59230     
Total params: 118,414
Trainable params: 118,414
Non-trainable params: 0
_________________________________________________________________
Model: "discriminator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 31 0                                            
_____________________________________________________________

In [6]:
model = spectralGAN(autoencoder=autoencoder, discriminator=discriminator)

model.compile(
    a_optimizer = a_optimizer,
    d_optimizer = d_optimizer,
    a_loss = autoencoder_loss,
    d_loss = discriminator_loss,
    metrics = [ PSNR() ]
)


In [7]:
model.fit(train_ds, epochs=100, validation_data=test_ds)

Epoch 1/2
9/9 [==============================] - 34s 1s/step - a_loss: 0.0130 - d_loss: 1.3489 - psnr: 19.1917 - val_a_loss: 0.0120 - val_d_loss: 1.3812 - val_psnr: 19.1048
Epoch 2/2
9/9 [==============================] - 10s 1s/step - a_loss: 0.0096 - d_loss: 1.2913 - psnr: 20.3816 - val_a_loss: 0.0091 - val_d_loss: 1.3793 - val_psnr: 20.1613


In [8]:
model.summary()

Model: "spectral_gan"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generator (Functional)       (None, 512, 512, 31)      118414    
_________________________________________________________________
discriminator (Functional)   (None, 129, 129, 1)       166913    
Total params: 285,327
Trainable params: 284,815
Non-trainable params: 512
_________________________________________________________________
